## Machine Learning | Sistema de Recomendação feito com KNN (K-nearest neighbors) 

O KNN classidica cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos. Se os vizinhos mais próximos forem na sua maioria de uma classe, a amostra em questão será classificada nesta categoria.

Para este projeto utilizei a base de dados "book-recommender-system' para criar um modelo de previsão de livros baseado nas avaliações dos usuários.



## Análise Exploratória das bases 

In [ ]:
books = pd.read_csv('/kaggle/input/book-recommender-system/BX-Books.csv',sep=';',encoding='latin-1',error_bad_lines=False)
display(books.head())

In [ ]:
users = pd.read_csv('/kaggle/input/book-recommender-system/BX-Users.csv',sep=';',encoding='latin-1',error_bad_lines=False)

In [ ]:
display(users)

In [ ]:
ratings = pd.read_csv('/kaggle/input/book-recommender-system/BX-Book-Ratings.csv', sep=';', encoding='latin-1', error_bad_lines=False)

In [ ]:
display(ratings)

In [ ]:
#limpeza de informações irrelevantes e renomear colunas
books.rename(columns= {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year','Publisher':'publisher'},inplace=True)
users.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'},inplace=True)
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'book_rating'}, inplace=True)

In [ ]:
books

In [ ]:
#Quantidade de ratings(avaliações) por usuário
ratings['user_id'].value_counts()

In [ ]:
#Usuários que tem mais de 200 avaliações, vamos nomear essa variável de x
x = ratings['user_id'].value_counts() > 200
display(x)

In [ ]:
#Quantidade de usuários que fizeram mais de 200 avaliações, vamos nomear essa variável de y
y = x[x].index
print(y.shape)


In [ ]:
y

In [ ]:
#Trazer somente usuários que avaliaram mais de 200 vezes 
ratings = ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings

In [ ]:
#Juntando Tabelas de Avaliação com a Tabela de livros com Join ou Merge, chave primária = ISBN
rating_books = ratings.merge(books, on = 'ISBN')
rating_books

In [ ]:
# Quantidade de vezes que o livro foi avaliado
number_ratings = rating_books.groupby('title')['book_rating'].count().reset_index()
number_ratings

In [ ]:
number_ratings.rename(columns={'book_rating':'number_of_ratings'}, inplace=True)
number_ratings

In [ ]:
#Tabela Final juntar a tabela de livros (rating_books) com as avaliações com a tabela de quantidade de avaliações por livros (number_ratings)
final_rating = rating_books.merge(number_ratings, on='title')
final_rating.drop(columns={'Image-URL-S', 'Image-URL-L', 'Image-URL-M'}, inplace=True)
final_rating

## Transformação da base em uma Matriz

As avaliações dos usuários serão usadas como variáveis para fazer a análise preditiva. 

In [ ]:
#filtrar livros que tenham mais de 50 avaliações 
final_rating = final_rating[final_rating['number_of_ratings'] > 50]
final_rating


In [ ]:
#Descartar valores duplicados
final_rating.loc[:,('user_id','title')]
final_rating


## Criar uma Matriz para fazer o modelo de recomendação

Transpor usuários x título x nota dos usuários para transformar os valores em variáveis

In [ ]:
# transposição dos usuários (user_id) em colunas:
book_pivot = rating_books.pivot_table(columns ='user_id', index='title', values='book_rating')
book_pivot.shape


In [ ]:
book_pivot

In [ ]:
#Transformar valores nulos em zero
book_pivot.fillna(0, inplace=True)
book_pivot

In [ ]:
#Transformar matriz em matriz esparça
from scipy.sparse import csr_matrix
book_sparce = csr_matrix(book_pivot)
book_sparce

## Dados prontos para a Modelagem 

Uso do algotiritmo KNN (K-nearest neighbors) 
O KNN tenta classificar cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos. Se os vizinhos mais próximos forem majoritariamente de uma classe, a amostra em questão será classificada nesta categoria.

## Criação do Modelo Preditivo 

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm ='brute')
model.fit(book_sparce) #treino 

## Novas Predições 

In [ ]:
book_pivot


In [ ]:
book_pivot.head(140)

In [ ]:
book_pivot.query('title = 10000 Dreams Interpreted')

In [ ]:
distances, sugestions = model.kneighbors(book_pivot.iloc[140 , :].values.reshape(1,-1))

In [ ]:
#para fazer a predição do modelo
for i in range(len(sugestions)):
    print(book_pivot.index[sugestions[i]])